# ECSE 415 Intro to Computer Vision

Assignment 3_2

Author: Mustafa Javed

Images provided by Prof James J. Clark

Credit: Most of this code was taken from Tutorial : cifar10_tutorial

In [1]:
#import libraries
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# define device to use the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


Use Pytorch class torchvision.datasets.MNIST to (down)load the dataset.
Use batch size of 32.

In [3]:
# Load the MNIST dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(( 0.5), ( 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Implement a CNN with the layers mentioned below.

In [4]:
#import libraries
import torch.nn as nn
import torch.nn.functional as F

#define a class that can be used to make an object of CNN
class Net(nn.Module):

    def __init__(self):
        #define constructor to initialise layers of CNN
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 4096)
        x = self.fc1(x)
        return x


net = Net()

Create an instance of SGD optimizer with learning rate of 0.001. Use the default setting for rest of the hyperparameters. Create an instance of categorical cross entropy criterion.

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

Train the CNN for 10 epochs.
Using a GPU for faster processing

In [6]:
net = net.to(device)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 500 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

print('Finished Training')

[1,   500] loss: 2.295
[1,  1000] loss: 2.256
[1,  1500] loss: 1.833
[2,   500] loss: 0.519
[2,  1000] loss: 0.467
[2,  1500] loss: 0.435
[3,   500] loss: 0.412
[3,  1000] loss: 0.394
[3,  1500] loss: 0.379
[4,   500] loss: 0.369
[4,  1000] loss: 0.342
[4,  1500] loss: 0.342
[5,   500] loss: 0.324
[5,  1000] loss: 0.315
[5,  1500] loss: 0.294
[6,   500] loss: 0.267
[6,  1000] loss: 0.257
[6,  1500] loss: 0.255
[7,   500] loss: 0.221
[7,  1000] loss: 0.209
[7,  1500] loss: 0.201
[8,   500] loss: 0.191
[8,  1000] loss: 0.165
[8,  1500] loss: 0.171
[9,   500] loss: 0.159
[9,  1000] loss: 0.144
[9,  1500] loss: 0.143
[10,   500] loss: 0.131
[10,  1000] loss: 0.131
[10,  1500] loss: 0.122
Finished Training


Predicts labels of the test images using the above trained CNN. Measure
and display classification accuracy.

In [7]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy of the network on the test images: " + str(100 * correct / total) + "%")

Accuracy of the network on the test images: 96.79%
